# 01. Configuração e Extração de Features

Este notebook cobre a configuração inicial do ambiente (Google Colab ou Local), montagem do Google Drive, criação de dataset de exemplo e execução do pipeline de extração de características de áudio em lote.

In [ ]:
# 1. Configuração do Ambiente
import os
import sys
import subprocess

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    print("Ambiente Google Colab detectado.")
    # Clone o repositório se não existir (ajuste a URL conforme necessário)
    if not os.path.exists('/content/TCC'):
        !git clone https://github.com/thieryw/TCC.git /content/TCC
    
    # Mudar diretório de trabalho
    os.chdir('/content/TCC')
    
    # Instalar dependências
    print("Instalando dependências...")
    !apt-get install -y ffmpeg
    !pip install -r requirements.txt
    !pip install python-multipart
    
    # Adicionar ao path
    sys.path.append('/content/TCC')
else:
    print("Ambiente Local detectado.")
    # Assumindo que você está na raiz do projeto
    if os.path.exists('app'):
        sys.path.append(os.getcwd())

print(f"Diretório atual: {os.getcwd()}")

In [ ]:
# 2. Importações e Setup do Sistema
from app.utils.colab import mount_drive, setup_colab_env
from app.domain.services.feature_extraction_service import AudioFeatureExtractionService
from app.domain.models.audio_data import AudioData
from app.domain.models.extraction_config import ExtractionConfig
import librosa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soundfile as sf

# Configurar ambiente Colab (se aplicável)
setup_colab_env()
mount_drive()

## 3. Aquisição e Preparação de Dados
Vamos criar um dataset sintético para demonstração, organizado em pastas `real` e `fake`.

In [ ]:
DATASET_RAW_DIR = "datasets/raw_sample"

def create_dummy_audio(filename, duration=1.0, sr=22050, freq=440):
    t = np.linspace(0, duration, int(duration*sr))
    y = np.sin(2*np.pi*freq*t)
    sf.write(filename, y, sr)

def create_dummy_dataset(base_dir, num_samples=10):
    classes = ['real', 'fake']
    for label in classes:
        class_dir = os.path.join(base_dir, label)
        os.makedirs(class_dir, exist_ok=True)
        for i in range(num_samples):
            # Frequências diferentes para diferenciar classes visualmente nos features
            freq = 440 if label == 'real' else 880 
            filename = os.path.join(class_dir, f"{label}_{i}.wav")
            if not os.path.exists(filename):
                create_dummy_audio(filename, freq=freq)
    print(f"Dataset dummy criado em {base_dir}")

create_dummy_dataset(DATASET_RAW_DIR, num_samples=5)

# Listar estrutura
for root, dirs, files in os.walk(DATASET_RAW_DIR):
    for file in files:
        print(os.path.join(root, file))

## 4. Extração de Features em Lote
Processar todo o dataset e preparar para treinamento (X, y).

In [ ]:
# Inicializar serviço
extractor_service = AudioFeatureExtractionService()
print("Serviço de extração inicializado.")

# Configuração
config = ExtractionConfig(
    mfcc=True,
    spectral_centroid=True,
    chroma=True,
    zero_crossing_rate=True
)

X = []
y = []
classes = {'real': 0, 'fake': 1}

print("Iniciando extração...")

for label_name, label_idx in classes.items():
    class_dir = os.path.join(DATASET_RAW_DIR, label_name)
    if not os.path.exists(class_dir): continue
    
    for file in os.listdir(class_dir):
        if not file.endswith('.wav'): continue
        
        file_path = os.path.join(class_dir, file)
        try:
            # Carregar e extrair
            audio_y, sr = librosa.load(file_path, sr=None, duration=3.0)
            # Garantir tamanho fixo para CNN simples (pad ou truncate)
            # Vamos fixar em ~128 frames para MFCC (aprox 3s)
            target_len = 22050 * 3 # 3 segundos
            if len(audio_y) < target_len:
                audio_y = np.pad(audio_y, (0, target_len - len(audio_y)))
            else:
                audio_y = audio_y[:target_len]
                
            audio_data = AudioData(audio=audio_y, sample_rate=sr)
            result = extractor_service.extract_features(audio_data, config)
            
            # Usar MFCC como feature principal para este exemplo
            # Shape típico: (n_mfcc, time_steps)
            feature = result.features.get('mfcc')
            
            if feature is not None:
                # Transpor para (time, n_mfcc) se necessário ou manter para CNN 2D
                # Vamos manter (n_mfcc, time) que é (20, 130) aprox
                # Ajustar shape para consistência exata
                desired_shape = (20, 130)
                if feature.shape[1] > desired_shape[1]:
                    feature = feature[:, :desired_shape[1]]
                elif feature.shape[1] < desired_shape[1]:
                    feature = np.pad(feature, ((0,0), (0, desired_shape[1] - feature.shape[1])))
                
                X.append(feature)
                y.append(label_idx)
                
        except Exception as e:
            print(f"Erro em {file}: {e}")

X = np.array(X)
y = np.array(y)

print(f"Extração concluída. X shape: {X.shape}, y shape: {y.shape}")

## 5. Visualização e Salvamento
Salvar o dataset processado no formato .npz para o notebook de treinamento.

In [ ]:
# Dividir em treino e validação (simples)
from sklearn.model_selection import train_test_split

if len(X) > 0:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    OUTPUT_DIR = "datasets/processed"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    output_path = os.path.join(OUTPUT_DIR, "dataset_processed.npz")
    np.savez(output_path, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val)
    print(f"Dataset salvo em: {output_path}")
    
    # Copiar para o Drive
    if is_colab() and os.path.exists('/content/drive/MyDrive'):
        DRIVE_PATH = "/content/drive/MyDrive/TCC_Features"
        os.makedirs(DRIVE_PATH, exist_ok=True)
        import shutil
        shutil.copy(output_path, DRIVE_PATH)
        print(f"Copiado para o Drive: {DRIVE_PATH}")
else:
    print("Nenhum dado extraído.")